In [25]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import random
from astropy.io import fits
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.table import Table, hstack

### Import the relevant files

In [17]:
hsc_cosmos_path= '/global/cscratch1/sd/damonge/HSC/HSC_processed/DEEP_COSMOS/DEEP_COSMOS_Catalog_i24.50.fits'
hdulist = fits.open(hsc_cosmos_path)
hsc_cosmos_data = hdulist[1].data
hsc_cosmos_columns= hdulist[1].columns

cosmos_path= '/global/cscratch1/sd/damonge/HSC/COSMOS2015_Laigle+_v1.1.fits'
hdulist = fits.open(cosmos_path)
cosmos_data = hdulist[1].data
cosmos_columns= hdulist[1].columns

### Cut COSMOS 30-band so they match HSC

See https://arxiv.org/pdf/1704.05988.pdf 3rd page on "COSMOS data" part 2, "Photo-z"

In [18]:
lim_indices = np.where((0.01 < cosmos_data['PHOTOZ']) & (9 > cosmos_data['PHOTOZ']) & (cosmos_data['TYPE'] == 0) & (cosmos_data['ZP_2'] < 0) & (cosmos_data['MASS_BEST'] > 7.5) 
            & (np.maximum(cosmos_data['ZPDF_H68'] - cosmos_data['ZPDF'], cosmos_data['ZPDF'] - cosmos_data['ZPDF_L68']) < 0.05*(1+cosmos_data['PHOTOZ']))
            & (cosmos_data['CHI2_BEST'] < cosmos_data['CHIS']) & (cosmos_data['CHI2_BEST']/cosmos_data['NBFILT'] < 5.))

cosmos_data_cut= cosmos_data[lim_indices] #length= 197640

/global/common/cori/software/python/3.6-anaconda-5.2/lib/python3.6/site-packages/ipykernel/__main__.py:3: RuntimeWarning: divide by zero encountered in true_divide
  app.launch_new_instance()


### Match the 30-band COSMOS to the HSC

In [27]:
cosmos_skycoord = SkyCoord(ra = cosmos_data_cut['ALPHA_J2000'] * u.deg, dec = cosmos_data_cut['DELTA_J2000']*u.deg)

hsc_skycoord = SkyCoord(ra = hsc_cosmos_data['ra'] * u.deg, dec = hsc_cosmos_data['dec'] * u.deg)
    
hsc_index, dist_2d, dist_3d = cosmos_skycoord.match_to_catalog_sky(hsc_skycoord)

In [28]:
hsc_match= hsc_cosmos_data[hsc_index]

### Make a fits file with this match

In [23]:
cosmos_data_cut_df= pd.DataFrame(cosmos_data_cut)
hsc_match_mag_df= pd.DataFrame(np.transpose([hsc_match['gcmodel_mag'], hsc_match['rcmodel_mag'], hsc_match['icmodel_mag'], hsc_match['zcmodel_mag'], hsc_match['ycmodel_mag']]),index=range(197640),columns=['gcmodel_mag','rcmodel_mag','icmodel_mag','zcmodel_mag','ycmodel_mag'] )

t1 = Table.from_pandas(cosmos_data_cut_df)
t2 = Table.from_pandas(hsc_match_mag_df)

In [170]:
matched_catalog= hstack([t1, t2])
matched_catalog.write('cosmos_matched_hsc.fits')

In [5]:
cosmos_matched_path= 'cosmos_matched_hsc.fits'
hdulist = fits.open(cosmos_matched_path)
cosmos_matched_data = hdulist[1].data
cosmos_matched_df= pd.DataFrame(cosmos_matched_data)